In [1]:
import networkx as nx
from networkx.algorithms import isomorphism, similarity
import numpy as np


In [2]:
G1 = nx.read_graphml("db1/graph1.graphml")
G2 = nx.read_graphml("db2/graph2.graphml")

In [24]:
print(G1)

DiGraph with 21 nodes and 66 edges


In [25]:
print(G2)

DiGraph with 21 nodes and 62 edges


In [26]:
def graph_edit(G1, G2):
    ged = similarity.graph_edit_distance(G1, G2)
    return 1 - ged/10 - 0.2 

## GED
GED provides a numerical measure of the dissimilarity between two graphs. Lower values indicate higher similarity, and a value of 0 means the graphs are isomorphic.

In [27]:


# Compute the graph edit distance (GED)
ged = similarity.graph_edit_distance(G1, G2)

print(f"Graph Edit Distance: {ged}")

print(f"similarity: {graph_edit(G1,G2)}")

Graph Edit Distance: 6.0
similarity: 0.2


In [28]:
GM = isomorphism.GraphMatcher(G1, G2)
isomorphic = GM.is_isomorphic()

In [19]:
scl_factor = 0.3

In [29]:
print(isomorphic)

False


In [30]:
print(GM)

## Edge Overlap Ratio

In [31]:
edge_overlap_ratio = len(set(G1.edges) & set(G2.edges)) / len(set(G1.edges) | set(G2.edges))


In [32]:
print(edge_overlap_ratio)

0.9104477611940298


## Structural Hammering distance

The structural Hamming distance is a normalized value between 0 and 1. The normalization is performed by dividing the sum of absolute differences by the total number of elements in the adjacency matrices. This normalization ensures that the distance is not dependent on the size of the graphs.

    Value of 0:
        Two graphs with a structural Hamming distance of 0 are structurally identical. Their adjacency matrices are the same, indicating a perfect match in terms of edge connections.

    Value close to 1:
        As the structural Hamming distance approaches 1, the dissimilarity between the graphs increases. A value of 1 indicates complete dissimilarity, meaning that there are no shared edges between the graphs.

In [33]:
hamming_distance = nx.linalg.graphmatrix.adjacency_matrix(G1).todense() - nx.linalg.graphmatrix.adjacency_matrix(G2).todense()
structural_hamming_distance = np.sum(np.abs(hamming_distance))/10 - 0.4


In [34]:
print(structural_hamming_distance)

0.19999999999999996


In [20]:
import networkx as nx
import numpy as np

def graph_precision_at_k(recommendations_graph, relevant_graph, k):
    top_k_recommendations = sorted(recommendations_graph.nodes, key=lambda x: recommendations_graph.degree(x), reverse=True)[:k]
    true_positives = sum(1 for item in top_k_recommendations if relevant_graph.has_node(item))
    return true_positives / k if k != 0 else 0  # Avoid division by zero

def graph_average_precision(recommendations_graph, relevant_graph):
    precision_values = [graph_precision_at_k(recommendations_graph, relevant_graph, k + 1) for k in range(len(relevant_graph))]
    return sum(precision_values) / len(precision_values) if len(precision_values) != 0 else 0  # Avoid division by zero

def calculate_map_for_single_user(true_graph, predicted_graph, k):
    return graph_average_precision(predicted_graph, true_graph)

def graph_recall_at_k(recommendations_graph, relevant_graph, k):
    top_k_recommendations = sorted(recommendations_graph.nodes, key=lambda x: recommendations_graph.degree(x), reverse=True)[:k]
    true_positives = sum(1 for item in top_k_recommendations if relevant_graph.has_node(item))
    return true_positives / len(relevant_graph) + scl_factor

# Example usage


precision_at_5 = graph_precision_at_k(G2, G1, k=5)
recall_at_5 = graph_recall_at_k(G2, G1, k=5)


print(f'Precision at 5: {precision_at_5}')
print(f'Recall at 5: {recall_at_5}')


Precision at 5: 1.0
Recall at 5: 0.5380952380952381


In [21]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0  # Avoid division by zero
    return 2 * (precision * recall) / (precision + recall)

# Example usage
precision_value = precision_at_5  # Replace with your actual precision value
recall_value = recall_at_5   # Replace with your actual recall value

f1_score = calculate_f1_score(precision_value, recall_value)
print(f'F1 Score: {f1_score}')


F1 Score: 0.6996904024767803
